In [ ]:
# Imports
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import os

# Load test CSV
test_df = pd.read_csv("/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv")
image_ids = test_df["image_id"].tolist()

# Label encoder (same order used during training)
soil_classes = ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
le = LabelEncoder()
le.fit(soil_classes)

# Image transform
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 4)
model.load_state_dict(torch.load("resnet_soil_model.pth", map_location=device))
model = model.to(device)
model.eval()

# Predict
predicted_labels = []
for img_id in image_ids:
    img_path = os.path.join("test_resized", img_id)
    try:
        image = Image.open(img_path).convert("RGB")
        image = test_transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            outputs = model(image)
            _, predicted_class = torch.max(outputs, 1)
            predicted_labels.append(predicted_class.item())
    except Exception as e:
        print(f"Failed to process {img_id}: {e}")
        predicted_labels.append(0)  # fallback to first class if error occurs

# Decode back to soil type
predicted_soil_types = le.inverse_transform(predicted_labels)

# Build submission DataFrame
submission_df = pd.DataFrame({
    "image_id": image_ids,
    "soil_type": predicted_soil_types
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)
print("✅ Submission file saved as submission.csv")

# Show sample
pd.read_csv("submission.csv").head()